In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from numpy import array

import warnings 
warnings.filterwarnings("ignore")

In [5]:
data = pd.read_excel(r'D:\MyData\chenpf8\Desktop\test_data.xlsx',sheet_name=0)
data

,id,interest,ClickTimes,LastTime,CommentTimes,collection,ScoreTimes
0,a001,a,3,2020-07-23,1,1,1
1,a001,b,4,2020-07-20,2,1,1
2,a001,c,1,2020-07-21,1,2,0
3,a001,d,2,2020-07-22,5,1,1
4,a001,e,4,2020-07-01,2,1,0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            5 non-null      object        
 1   interest      5 non-null      object        
 2   ClickTimes    5 non-null      int64         
 3   LastTime      5 non-null      datetime64[ns]
 4   CommentTimes  5 non-null      int64         
 5   collection    5 non-null      int64         
 6   ScoreTimes    5 non-null      int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 408.0+ bytes


In [10]:
data['LastTime'] =   data['LastTime'] - datetime.datetime.now()
data['LastTime'].dt.total_seconds()  # 转换为相差 秒数
data['LastTime'] = data['LastTime'].dt.total_seconds()/(3600*24)  #转为相差天数
data

,id,interest,ClickTimes,LastTime,CommentTimes,collection,ScoreTimes
0,a001,a,3,-123.739605,1,1,1
1,a001,b,4,-126.739605,2,1,1
2,a001,c,1,-125.739605,1,2,0
3,a001,d,2,-124.739605,5,1,1
4,a001,e,4,-145.739605,2,1,0


In [12]:
data_1 = data[['ClickTimes', 'LastTime','CommentTimes',
       'collection', 'ScoreTimes']]
data_1

,ClickTimes,LastTime,CommentTimes,collection,ScoreTimes
0,3,-123.739605,1,1,1
1,4,-126.739605,2,1,1
2,1,-125.739605,1,2,0
3,2,-124.739605,5,1,1
4,4,-145.739605,2,1,0


In [13]:
data_1 = (data_1 - data_1.min()) / (data_1.max() - data_1.min())
data_1

,ClickTimes,LastTime,CommentTimes,collection,ScoreTimes
0,0.666667,1.000000,0.00,0.0,1.0
1,1.000000,0.863636,0.25,0.0,1.0
2,0.000000,0.909091,0.00,1.0,0.0
3,0.333333,0.954545,1.00,0.0,1.0
4,1.000000,0.000000,0.25,0.0,0.0


In [21]:
m,n = data_1.shape
k = 1.0 / math.log(m)
lnp_arr = array([[None] * n for i in range(m)])

arr_1 = array(data_1)
for i in range(m):
    for j in range(n):
        if arr_1[i][j] == 0:
            lnp = 0
        else:
            p = arr_1[i][j] / arr_1.sum(axis=0)[j]
            lnp = math.log(p) * p * (-k)
#         print(lnp)
        lnp_arr[i][j] = lnp
res = pd.DataFrame(lnp_arr)
ans = res.sum(axis=0)      #  熵值
ans

0    0.814436
1    0.860428
2    0.539047
3    0.000000
4    0.682606
dtype: float64

In [24]:
weight = [[None] * 1 for i in range(m)]
for i in range(m):
               sum1 = 1 * len(ans) - sum(ans)
               weight[i] = ((1-ans[i]) / sum1)
weight

[0.08821766390953817,
 0.0663526768364859,
 0.21913786582597738,
 0.4754021089264458,
 0.15088968450155274]

### 第一种思路，直接用原始数据

In [35]:
data_use = data[['ClickTimes', 'LastTime','CommentTimes',
       'collection', 'ScoreTimes']]

weight_arr = array(weight)
data_use_arr = array(data_use)

result = [[0] * 1 for i in range(m)]
for i in range(m):
    for j in range(n):
        res = data_use_arr[i][j] * weight_arr[j]
        result[i] += res
result1 = pd.DataFrame(result)
result1 + 10

,0
0,2.899629
1,3.007926
2,2.915000
3,3.621610
4,1.596336


###  第二种思路，用归一化后的数据(这个可能是有点问题的）

In [36]:
data_use = data_1

weight_arr = array(weight)
data_use_arr = array(data_use)

result = [[0] * 1 for i in range(m)]
for i in range(m):
    for j in range(n):
        res = data_use_arr[i][j] * weight_arr[j]
        result[i] += res
result2 = pd.DataFrame(result)
result2

,0
0,0.276054
1,0.351196
2,0.535723
3,0.462770
4,0.143002


In [9]:
import pandas as pd

a=['小红','小绿','小蓝','小白','小青','小紫','小粉','小傻','小红','小绿','小蓝','小白','小青','小紫','小粉','小傻','小红','小绿','小蓝','小白','小青','小紫','小粉','小傻']
b=['语文','语文','语文','语文','语文','语文','语文','语文','数学','数学','数学','数学','数学','数学','数学','数学','英语','英语','英语','英语','英语','英语','英语','英语']
c=[97,65,23,43,67,23,55,98,56,45,67,78,98,45,87,65,67,23,55,98,56,45,67,78]
len(a),len(b),len(c)

df=pd.DataFrame({'name':a,'kemu':b,'score':c})

df2=df.sort_values(['kemu','score','name'], ascending=[1, 0,1])

# df2

df2['rn']=df2.groupby(['kemu']).rank(method='first',ascending =0)['score']

df2[df2['rn']<=5]



,name,kemu,score,rn
12,小青,数学,98,1.0
14,小粉,数学,87,2.0
11,小白,数学,78,3.0
10,小蓝,数学,67,4.0
15,小傻,数学,65,5.0
19,小白,英语,98,1.0
23,小傻,英语,78,2.0
22,小粉,英语,67,3.0
16,小红,英语,67,4.0
20,小青,英语,56,5.0


In [11]:
20/3

6.666666666666667